<a href="https://colab.research.google.com/github/anoted/Assignment-4/blob/main/scripts/assignment_4_part_III.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part III
Using the previous two tutorials, please answer the following using an encorder-decoder approach and an LSTM compared approach.

Please create a transformer-based classifier for English name classification into male or female.

There are several datasets for name for male or female classification. In subseuqent iterations, this could be expanded to included more classifications.

Below is the source from NLTK, which only has male and female available but could be used for the purposes of this assignment.

```
names = nltk.corpus.names
names.fileids()
['female.txt', 'male.txt']
male_names = names.words('male.txt')
female_names = names.words('female.txt')
[w for w in male_names if w in female_names]
['Abbey', 'Abbie', 'Abby', 'Addie', 'Adrian', 'Adrien', 'Ajay', 'Alex', 'Alexis',
'Alfie', 'Ali', 'Alix', 'Allie', 'Allyn', 'Andie', 'Andrea', 'Andy', 'Angel',
'Angie', 'Ariel', 'Ashley', 'Aubrey', 'Augustine', 'Austin', 'Averil', ...]
```

In [1]:
!pip install keras-nightly

In [45]:
import tensorflow as tf
import string
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras import ops
from keras import layers
import random
import re
import nltk
nltk.download('names')

[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!


True

In [2]:
def get_angles(pos, i, d_model):
  angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
  return pos * angle_rates

def positional_encoding(position, d_model):
  angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                          np.arange(d_model)[np.newaxis, :],
                          d_model)

  # apply sin to even indices in the array; 2i
  angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

  # apply cos to odd indices in the array; 2i+1
  angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

  pos_encoding = angle_rads[np.newaxis, ...]

  return tf.cast(pos_encoding, dtype=tf.float32)



___


In [58]:
vocab_size = 26
maxlen = 8
letters = string.ascii_lowercase
names = nltk.corpus.names
#names.fileids()

male_names = names.words('male.txt')
female_names = names.words('female.txt')

random.shuffle(male_names)
random.shuffle(female_names)


x_train = []
y_train = []

for name in male_names[:len(male_names)-200]:
    name = re.sub(r'[^A-Za-z]', '', name) # letter only
    name_letters = list(name.lower())     # lower and list
    x_name = []
    for letter in name_letters:
        #print(letter)
        x_name.append(letters.index(letter)+1) # index as alphabet index 1-26
    x_train.append(x_name)
    y_train.append(1)                     # 1 -> male

for name in female_names[:len(female_names)-200]:
    name = re.sub(r'[^A-Za-z]', '', name) # letter only
    name_letters = list(name.lower())     # lower and list
    x_name = []
    for letter in name_letters:
        #print(letter)
        x_name.append(letters.index(letter)+1) # index as alphabet index 1-26
    x_train.append(x_name)
    y_train.append(0)                     # 1 -> female

x_val = []
y_val = []

for name in male_names[len(male_names)-200:]:
    name = re.sub(r'[^A-Za-z]', '', name) # letter only
    name_letters = list(name.lower())     # lower and list
    x_name = []
    for letter in name_letters:
        #print(letter)
        x_name.append(letters.index(letter)+1) # index as alphabet index 1-26
    x_val.append(x_name)
    y_val.append(1)                     # 1 -> male

for name in female_names[len(female_names)-200:]:
    name = re.sub(r'[^A-Za-z]', '', name) # letter only
    name_letters = list(name.lower())     # lower and list
    x_name = []
    for letter in name_letters:
        #print(letter)
        x_name.append(letters.index(letter)+1) # index as alphabet index 1-26
    x_val.append(x_name)
    y_val.append(0)                     # 1 -> female

# print(x_train)
# print(x_val)
x_train = keras.utils.pad_sequences(x_train, maxlen=maxlen)
x_val = keras.utils.pad_sequences(x_val, maxlen=maxlen)


[[14, 9, 3, 3, 15, 12, 15], [1, 14, 20, 9, 14], [4, 1, 22, 9, 4, 19, 15, 14], [23, 1, 9, 14], [5, 18, 14, 25], [12, 5, 9, 6], [3, 12, 1, 21, 4, 9, 15], [13, 1, 18, 3, 15, 19], [20, 9, 20, 21, 19], [13, 1, 14, 6, 18, 5, 4], [3, 18, 9, 19], [23, 5, 2, 2], [3, 8, 1, 18, 12, 5, 20, 15, 14], [22, 9, 2, 8, 21], [20, 18, 1, 3, 5, 25], [6, 12, 5, 13], [1, 18, 5, 12], [4, 15, 21, 7, 12, 1, 19], [12, 5, 15, 14, 9, 4, 1, 19], [19, 9, 4, 14, 5, 25], [6, 18, 5, 5, 13, 15, 14], [8, 5, 18, 3, 21, 12, 5, 19], [19, 8, 5, 16, 16, 5, 18, 4], [4, 9, 14, 15], [1, 12, 1, 9, 14], [7, 9, 15, 18, 7, 9], [4, 1, 2, 14, 5, 25], [10, 15, 20, 8, 1, 13], [20, 9, 20, 15, 19], [6, 18, 1, 26, 9, 5, 18], [3, 25, 18, 9, 12, 12, 5], [1, 12, 5, 19, 19, 1, 14, 4, 18, 15], [7, 9, 1, 3, 15, 13, 15], [18, 15, 21, 18, 11, 5], [13, 21, 19, 20, 1, 6, 1], [7, 21, 19, 20, 1, 22], [12, 15, 3, 11, 23, 15, 15, 4], [10, 15, 5], [13, 9, 3, 11], [7, 18, 1, 8, 1, 13], [8, 1, 25, 4, 5, 14], [23, 15, 18, 20, 8, 9, 14, 7, 20, 15, 14], [2, 5,

In [5]:
embed_dim = 32  # Embedding size for each token
num_heads = 2   # Number of attention heads
ff_dim = 32     # Hidden layer size in feed forward network inside transformer

inputs = layers.Input(shape=(maxlen,))
embedding_layer = TokenAndPositionEmbedding(maxlen, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)

model = keras.Model(inputs=inputs, outputs=outputs)


In [6]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
history = model.fit(
    x_train, y_train, batch_size=32, epochs=2, validation_data=(x_val, y_val)
)


Epoch 1/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 141s 166ms/step - accuracy: 0.6991 - loss: 0.5296 - val_accuracy: 0.8792 - val_loss: 0.2829
Epoch 2/2
782/782 ━━━━━━━━━━━━━━━━━━━━ 122s 141ms/step - accuracy: 0.9264 - loss: 0.1929 - val_accuracy: 0.8719 - val_loss: 0.3141


In [13]:
zx = model.predict(keras.utils.pad_sequences([[1,1,1,1,1]], maxlen=maxlen))
zx

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step


array([[0.70086527, 0.29913476]], dtype=float32)

In [14]:
### START CODE HERE (REPLACE INSTANCES OF 'None' with your code) ###

names = nltk.corpus.names
#names.fileids()

male_names = names.words('male.txt')
female_names = names.words('female.txt')
#[w for w in male_names if w in female_names]


### END CODE HERE ###

In [16]:
len(female_names)

5001

# References
1. https://arxiv.org/pdf/2102.03692.pdf
2. https://alvinntnu.github.io/NTNU_ENC2045_LECTURES/exercise/13-attention.html
3. https://towardsdatascience.com/deep-learning-gender-from-name-lstm-recurrent-neural-networks-448d64553044
4. https://www.nltk.org/book/ch02.html#sec-lexical-resources